In [6]:
%matplotlib inline
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras import losses
import h5py

DATA_FILE = 'data.h5'

In [7]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    labels_testing = list(f['label_testing'])

In [22]:

activation = 'relu'

input_shape = (128, 128, 1)
pic_input = Input(shape=input_shape)

img_stack = Conv2D(32, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(128, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(123, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(128, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(64, activation=activation, name='dense1')(dense_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense2')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss=losses.mean_absolute_error)

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 128)       0         
__________

In [24]:
from generator import DriveDataGenerator

train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=input_shape)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=input_shape)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=300, verbose=1, callbacks=[plateau_callback])

Epoch 1/300
100/100 [==============================] - 10s 100ms/step - loss: 0.9587 - val_loss: 1.0023
Epoch 2/300
100/100 [==============================] - 8s 81ms/step - loss: 0.9951 - val_loss: 0.9989
Epoch 3/300
100/100 [==============================] - 8s 80ms/step - loss: 0.9911 - val_loss: 1.0009
Epoch 4/300
100/100 [==============================] - 8s 79ms/step - loss: 0.9982 - val_loss: 0.9692
Epoch 5/300
100/100 [==============================] - 8s 83ms/step - loss: 0.8194 - val_loss: 0.2896
Epoch 6/300
100/100 [==============================] - 9s 91ms/step - loss: 0.2819 - val_loss: 0.2570
Epoch 7/300
100/100 [==============================] - 9s 90ms/step - loss: 0.2527 - val_loss: 0.2266
Epoch 8/300
100/100 [==============================] - 9s 89ms/step - loss: 0.2410 - val_loss: 0.2204
Epoch 9/300
100/100 [==============================] - 8s 77ms/step - loss: 0.2328 - val_loss: 0.2209
Epoch 10/300
100/100 [==============================] - 8s 78ms/step - loss: 0.2

100/100 [==============================] - 7s 67ms/step - loss: 0.1667 - val_loss: 0.1551
Epoch 78/300
100/100 [==============================] - 7s 67ms/step - loss: 0.1648 - val_loss: 0.1522
Epoch 79/300
100/100 [==============================] - 7s 65ms/step - loss: 0.1682 - val_loss: 0.1577
Epoch 80/300
100/100 [==============================] - 6s 64ms/step - loss: 0.1670 - val_loss: 0.1505
Epoch 81/300
100/100 [==============================] - 6s 58ms/step - loss: 0.1673 - val_loss: 0.1535
Epoch 82/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1697 - val_loss: 0.1515
Epoch 83/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1678 - val_loss: 0.1545
Epoch 84/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1642 - val_loss: 0.1564
Epoch 85/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1697 - val_loss: 0.1543
Epoch 86/300
100/100 [==============================] - 6s 59ms/step - loss: 0.1638 - 

100/100 [==============================] - 7s 66ms/step - loss: 0.1546 - val_loss: 0.1473
Epoch 157/300
100/100 [==============================] - 6s 58ms/step - loss: 0.1550 - val_loss: 0.1441
Epoch 158/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1569 - val_loss: 0.1454
Epoch 159/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1585 - val_loss: 0.1457
Epoch 160/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1533 - val_loss: 0.1450
Epoch 161/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1545 - val_loss: 0.1500
Epoch 162/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1556 - val_loss: 0.1464
Epoch 163/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1571 - val_loss: 0.1468
Epoch 164/300
100/100 [==============================] - 6s 58ms/step - loss: 0.1542 - val_loss: 0.1401
Epoch 165/300
100/100 [==============================] - 6s 55ms/step - loss: 

Epoch 235/300
100/100 [==============================] - 6s 58ms/step - loss: 0.1455 - val_loss: 0.1378
Epoch 236/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1467 - val_loss: 0.1396
Epoch 237/300
100/100 [==============================] - 6s 60ms/step - loss: 0.1449 - val_loss: 0.1443
Epoch 238/300
100/100 [==============================] - 6s 58ms/step - loss: 0.1445 - val_loss: 0.1397
Epoch 239/300
100/100 [==============================] - 6s 59ms/step - loss: 0.1437 - val_loss: 0.1386
Epoch 240/300
100/100 [==============================] - 6s 57ms/step - loss: 0.1436 - val_loss: 0.1448
Epoch 241/300
100/100 [==============================] - 6s 56ms/step - loss: 0.1472 - val_loss: 0.1400
Epoch 242/300
100/100 [==============================] - 6s 56ms/step - loss: 0.1456 - val_loss: 0.1407
Epoch 243/300
100/100 [==============================] - 6s 56ms/step - loss: 0.1427 - val_loss: 0.1361
Epoch 244/300
100/100 [==============================] - 6s 56ms

In [25]:
save_model(model, 'model.h5')